# Test Scan 53471

## Data comparison between two MongoDB databases

In [1]:
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# make graphics inline
%matplotlib inline

from databroker import Broker

In [2]:
scan_id = 53471

## databrokers and headers

In [3]:
db_new = Broker.named('mdb01-new')
hdr_new = db_new[scan_id]
db_1 = Broker.named('ca1-1')
hdr_1 = db_1[scan_id]

## run_start, run_stop, event_descriptors, and events

In [4]:
from math import isnan
def clean_data(t):
    if t[0] == 'event': 
        for k in list(t[1]['data'].keys()):
            v = t[1]['data'][k]
            if type(v) != str:
                if isnan(v):
                    t[1]['data'][k] = 'nan'   
    return t

In [5]:
doc_gen_new = db_new.get_documents(hdr_new, fill=False)
doc_gen_1 = db_1.get_documents(hdr_1, fill=False)

results = []
for x, y in zip(doc_gen_new, doc_gen_1):
    x = clean_data(x)
    y = clean_data(y)
    results.append(x == y)

In [6]:
print(any(results), len(results))

True 10006


## resource collection

In [7]:
res_uids_new = list(db_new.get_resource_uids(hdr_new))
res_uids_1 = list(db_1.get_resource_uids(hdr_1))

In [8]:
res_uids_new == res_uids_1

True

In [9]:
results = []
for uid in res_uids_new:
    res_doc_new = db_new.reg.resource_given_uid(uid)
    res_doc_1 = db_1.reg.resource_given_uid(uid)   
    results.append(res_doc_new == res_doc_1)

In [10]:
print(any(results), len(results))

True 2


## datum collection

In [11]:
results = []
for res_uid in res_uids_new:
    datum_gen_new = db_new.reg.datum_gen_given_resource(res_uid)
    datum_gen_1 = db_1.reg.datum_gen_given_resource(res_uid)
    for datum_new, datum_1 in zip(datum_gen_new, datum_gen_1):
        datum_new.pop('_id')
        datum_1.pop('_id')
        results.append(datum_new == datum_1)

In [12]:
print(any(results), len(results))

True 40000
